<a href="https://colab.research.google.com/github/pinedance/gym-AI-NLP/blob/main/fine_tune_BERT_%ED%95%9C%EB%AC%B8_%EC%9E%90%EB%8F%99_%ED%91%9C%EC%A0%90_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고문헌 자동 표점 테스트

source
* [raynardj/classical-chinese-punctuation-guwen-biaodian](https://huggingface.co/raynardj/classical-chinese-punctuation-guwen-biaodian)

REF
* [BertForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.BertForTokenClassification)

In [1]:
%%bash
# pip install transformers

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [3]:
model_name = "raynardj/classical-chinese-punctuation-guwen-biaodian"
tokenizer = AutoTokenizer.from_pretrained( model_name )
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
def clean_text( text ):
    _text = text + ""
    _text = _text.replace( ",", "，" )
    _text = _text.replace( ".", "。" )
    _text = _text.replace(" ", "")
    return _text


In [23]:
def split_text_to_src_tgt( text ):
    _text_list = list()
    for t in text:
        if t in ["，", "。"]:
            _text_list.append(t)
        else:
            _text_list.append( t )
            _text_list.append( "O")

    raw_text_new = "".join(_text_list)
    raw_text_new = raw_text_new.replace("O，", "，")
    raw_text_new = raw_text_new.replace("O。", "。")

    src, tgt = list(), list()
    for i, c in enumerate( raw_text_new):
        if i % 2 == 0:
            src.append( c )
        else:
            tgt.append( c )
    return "".join( src ), "".join( tgt )


In [12]:
new_inputs = [
    "凡濕病, 忌不得以火攻幷轉利之. 若濕家下之, 則額上汗出, 微喘, 小便不利者, 死, 下利不止者亦, 死.",
    "治濕不得猛發汗, 及灼艾灸之.",
    "濕病誤下, 則爲喘噦, 誤汗, 則發痓而死.",
    "濕家不可汗, 汗之則發痓, 發痓者斃. 又不可下, 下之則額汗胸滿, 微喘而噦, 小便淋閉, 難以有瘳也.",
    "太陽病, 發汗太多, 因致痓. 濕家大發汗, 亦作痓. 盖汗太多則亡陽, 不能養筋, 故筋脉緊急而成痓. 其證身熱足冷, 頸項强急, 惡寒, 時頭熱, 面赤目赤, 獨頭面搖, 卒口噤, 背反張者, 是也. 亦名破傷風."
]


In [24]:
text_cleaned = [ clean_text(text) for text in new_inputs ]
text_cleaned

['凡濕病，忌不得以火攻幷轉利之。若濕家下之，則額上汗出，微喘，小便不利者，死，下利不止者亦，死。',
 '治濕不得猛發汗，及灼艾灸之。',
 '濕病誤下，則爲喘噦，誤汗，則發痓而死。',
 '濕家不可汗，汗之則發痓，發痓者斃。又不可下，下之則額汗胸滿，微喘而噦，小便淋閉，難以有瘳也。',
 '太陽病，發汗太多，因致痓。濕家大發汗，亦作痓。盖汗太多則亡陽，不能養筋，故筋脉緊急而成痓。其證身熱足冷，頸項强急，惡寒，時頭熱，面赤目赤，獨頭面搖，卒口噤，背反張者，是也。亦名破傷風。']

In [35]:
src_text, tgt_text = list( zip( *[ split_text_to_src_tgt( text) for text in text_cleaned ] ) )

In [36]:
src_text

('凡濕病忌不得以火攻幷轉利之若濕家下之則額上汗出微喘小便不利者死下利不止者亦死',
 '治濕不得猛發汗及灼艾灸之',
 '濕病誤下則爲喘噦誤汗則發痓而死',
 '濕家不可汗汗之則發痓發痓者斃又不可下下之則額汗胸滿微喘而噦小便淋閉難以有瘳也',
 '太陽病發汗太多因致痓濕家大發汗亦作痓盖汗太多則亡陽不能養筋故筋脉緊急而成痓其證身熱足冷頸項强急惡寒時頭熱面赤目赤獨頭面搖卒口噤背反張者是也亦名破傷風')

In [37]:
tgt_text

('OO，OOOOOOOOO。OOOO，OOOO，O，OOOO，，OOOOO，。',
 'OOOOOO，OOOO。',
 'OOO，OOO，O，OOOO。',
 'OOOO，OOOO，OOO。OOO，OOOOOO，OOO，OOO，OOOO。',
 'OO，OOO，OO。OOOO，OO。OOOOOO，OOO，OOOOOOO。OOOOO，OOO，O，OO，OOO，OOO，OO，OOO，O。OOOO。')

In [5]:
inputs = tokenizer( example_input, add_special_tokens=False )

In [9]:
# tokenizer.tokenize( example_input )

['郡',
 '邑',
 '置',
 '夫',
 '子',
 '庙',
 '于',
 '学',
 '以',
 '[UNK]',
 '时',
 '释',
 '奠',
 '盖',
 '自',
 '唐',
 '贞',
 '观',
 '以',
 '来',
 '未',
 '之',
 '或',
 '改',
 '我',
 '宋',
 '有',
 '天',
 '下',
 '因',
 '其',
 '制',
 '而',
 '损',
 '益',
 '之',
 '姑',
 '苏',
 '当',
 '浙',
 '右',
 '要',
 '区',
 '规',
 '模',
 '尤',
 '大',
 '更',
 '建',
 '炎',
 '戎',
 '马',
 '荡',
 '然',
 '无',
 '遗',
 '虽',
 '修',
 '学',
 '宫',
 '于',
 '荆',
 '榛',
 '瓦',
 '砾',
 '之',
 '余',
 '独',
 '殿',
 '宇',
 '未',
 '遑',
 '议',
 '也',
 '每',
 '春',
 '秋',
 '展',
 '礼',
 '于',
 '斋',
 '庐',
 '已',
 '则',
 '置',
 '不',
 '问',
 '殆',
 '为',
 '阙',
 '典',
 '今',
 '[UNK]',
 '文',
 '阁',
 '直',
 '学',
 '士',
 '括',
 '苍',
 '梁',
 '公',
 '来',
 '牧',
 '之',
 '明',
 '年',
 '实',
 '绍',
 '兴',
 '十',
 '有',
 '一',
 '[UNK]',
 '也',
 '二',
 '月',
 '上',
 '丁',
 '修',
 '祀',
 '既',
 '毕',
 '乃',
 '[UNK]',
 '然',
 '自',
 '咎',
 '揖',
 '诸',
 '生',
 '而',
 '告',
 '之',
 '曰',
 '天',
 '子',
 '不',
 '以',
 '汝',
 '嘉',
 '为',
 '不',
 '肖',
 '俾',
 '再',
 '守',
 '兹',
 '土',
 '顾',
 '治',
 '民',
 '事',
 '神',
 '皆',
 '守',
 '之',
 '职',
 '惟',
 '是',
 '夫',
 '子',
 '之',
 '祀',


In [ ]:
with torch.no_grad():
    logits = model( **inputs ).logits

In [ ]:
predicted_token_class_ids = logits.argmax(-1)

In [ ]:
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

In [ ]:
list( set( predicted_tokens_classes ) )

In [ ]:
labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)

0.09

In [ ]:
def build_output( input_text, predicted_class ):
    if len(input_text) != len(predicted_class):
        print( "입력 텍스트와 결과 텍트트의 길이가 서로 다릅니다. ")
        return None
    output_list = list()
    for a, b in zip( input_text, predicted_class ):
        output_list.append( a )
        if b != "O":
            output_list.append( b )
    rst = "".join( output_list )
    rst = rst.replace("", "")
    return rst

In [ ]:
build_output( example_input, predicted_tokens_classes )

'郡邑，置夫子庙于学，以嵗时释奠。盖自唐。贞观以来，未之或改。我宋有天下因其制而损益之。姑苏当浙右要区，规模尤大，更建炎戎马，荡然无遗。虽修学宫于荆榛瓦砾之余，独殿宇未遑议也。每春秋展礼于斋庐，已则置不问，殆为阙典。今寳文阁直学士括苍梁公来牧之。明年，实绍兴十有一禩也。二月，上丁修祀既毕，乃愓然自咎，揖诸生而告之曰"天子不以汝嘉为不肖，俾再守兹土，顾治民事神，皆守之职。惟是夫子之祀，教化所基，尤宜严且谨。而拜跪荐祭之地，卑陋乃尔！其何以掲防妥灵？汝嘉不敢避其责。曩常去此弥年，若有所负，尚安得以罢輭自恕，复累后人乎！他日或克就绪，愿与诸君落之。于是谋之，僚吏搜故府，得遗材千枚，取赢资以给其费。鸠工庀役，各举其任。嵗月讫，工民不与知像设礼器，百用具修。至于堂室。廊序。门牖。垣墙，皆一新之。'